In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import json
import seaborn as sns

In [15]:
raw_communities = pd.read_csv("../Data/communities.csv")
raw_egm = pd.read_csv("../Data/EGM.csv")
raw_housingPrice = pd.read_csv("../Data/Houses-by-suburb.csv")
raw_offences = pd.read_csv("../Data/LGA_Offences_Table1.csv")

communities = raw_communities.copy()
egm = raw_egm.copy()
housingPrice = raw_housingPrice.copy()
offences = raw_offences.copy()

In [16]:
# Clean the LGA of the 3 tables (Housing price is special)
def clean_lga_tables(communities, egm, offences):
    def clean_lga(lga: str):
        lga = lga.strip().lower()
        lga = lga.replace('-', '')
        replaces = {
            'moreland': 'merribek'
        }
        lga = ''.join([replaces.get(word, word) for word in lga.split() if word not in ['of', 'city', 'rural', 'shire', 'borough']])
        # Remove the bracket from communities
        lga = lga.split('(')[0]
        return lga
    
    communities["LGA"] = communities["LGA"].apply(clean_lga)
    egm["LGA Name"] = egm["LGA Name"].apply(clean_lga)
    offences["Local Government Area"] = offences["Local Government Area"].apply(clean_lga)

In [17]:
# # Try to scrape the government website
# # Get the list of 79 LGA (local councils)
# def getLGAFromCouncilLink(url: str):
#     rePattern = r'/know-your-council-(.*)'
#     councilName = re.match(rePattern, url).group(1)
#     councilName = councilName.strip().lower()
#     words = councilName.split('-')
#     lga = ''.join([word for word in words if word not in ['of', 'city', 'rural', 'shire', 'borough', 'council']])
#     return lga

# prefixUrl = "https://www.vic.gov.au"
# page = requests.get("https://www.vic.gov.au/know-your-council")
# soup = BeautifulSoup(page.text, 'html.parser')

# councilList = soup.find("ul", {"class": 'vic-kyc__councils-list rpl-grid'})
# allLinksToCouncil = councilList.find_all("a")
# lgaLocality = {}

# for anchorTag in allLinksToCouncil:
#     link = anchorTag["href"]
#     lga = getLGAFromCouncilLink(link)

#     page = requests.get(prefixUrl + link)
#     localitySoup = BeautifulSoup(page.text, 'html.parser')
#     localityHeader = localitySoup.find(id="localities")
#     localityParagraph = localityHeader.find_next("p")
#     localityText = localityParagraph.text.strip(" .").replace(" and ", ", ")
#     locailityList = localityText.split(", ")

#     lgaLocality[lga] = locailityList

# # with open("lgaLocality.json", "w") as outFile:
# #     json.dump(lgaLocality, outFile, indent=4)

# lgaLocality


In [18]:
# # Try using the scraped data to map locality to LGA
# # There are missing localities, so mainly use this to find the missing localities
# with open("../lgaLocality.json", "r") as file:
#     lgaLocality = json.load(file)

# def clean_communities(community: str):
#     community = community.strip().lower()
#     # Strip off any brackets at the end
#     community = community.split("(")[0]
#     # Rip off directions
#     words = list(community.split())
#     for i in range(len(words)):
#         if words[i] in ("north", "south", "east", "west", "central"):
#             words[i] = ""
#     return ''.join(words)


# housingPrice["Locality"] = housingPrice["Locality"].apply(clean_communities)

# mapDictionary = {}
# cannotMap = []

# for housingPriceCommunity in housingPrice["Locality"]:
#     mapped = False
#     for lga in lgaLocality:
#         for community in lgaLocality[lga]:
#             community = clean_communities(community)
#             if housingPriceCommunity in community:
#                 mapDictionary[housingPriceCommunity] = (community, lga)
#                 mapped = True
#                 break
#         if mapped:
#             break
#     if not mapped:
#         cannotMap.append(housingPriceCommunity)

# print(mapDictionary)
# print(cannotMap)
# print(mapDictionary['osborne'])


In [19]:
# # Compute the full list to convert community to LGA
# def clean_communities(community: str):
#     community = community.strip().lower()
#     # Strip off any brackets at the end
#     community = community.split("(")[0]
#     # Rip off directions
#     words = list(community.split())
#     for i in range(len(words)):
#         if words[i] in ("north", "south", "east", "west", "central"):
#             words[i] = ""
#     return ''.join(words)

# with open("../lgaLocality.json", "r") as file:
#     lgaLocality = json.load(file)

# handMapLocalityToLGA = {'aintree': 'melton', 
#                       'balcombe': 'wodonga', 
#                       'benalla': 'benalla', 
#                       'bonniebrook': 'melton', 
#                       'botanicridge': 'casey', 
#                       'capelsound': 'morningtonpeninsula', 
#                       'cobblebank': 'melton', 
#                       'coonanshill': 'merribek', 
#                       'danyo': 'mildura', 
#                       'deanside': 'melton', 
#                       'deepdene': 'boroondara', 
#                       'eastwood': 'eastgippsland', 
#                       'fiveways': 'latrobe', 
#                       'fraserrise': 'melton', 
#                       'glenwaverley': 'monash', 
#                       'harkness': 'melton', 
#                       'jolimont': 'greatershepparton', 
#                       'laralake': 'greatergeelong', 
#                       'lucas': 'ballarat', 
#                       'manorlakes': 'wyndham', 
#                       'mansfield': 'mansfield', 
#                       'mckenziehill': 'mountalexander', 
#                       'merindapark': 'casey', 
#                       'pattersongardens': 'brimbank', 
#                       'sanctuarylakes': 'wyndham', 
#                       'sthelena': 'banyule', 
#                       'strathtulloh': 'melton', 
#                       'thornhillpark': 'melton', 
#                       'weirviews': 'melton', 
#                       'westall': 'greaterdandenong', 
#                       'westgarth': 'nillumbik', 
#                       'wimbledonheights': 'basscoast', 
#                       'wintervalley': 'ballarat', 
#                       'woolamaiwaters': 'basscoast'}
# localityToLGA = {}
# for community in mapDictionary:
#     localityToLGA[community] = mapDictionary[community][1]

# for community in handMapLocalityToLGA:
#     localityToLGA[community] = handMapLocalityToLGA[community]
# with open("../localityToLGA.json", "w") as outFile:
#     json.dump(localityToLGA, outFile, indent=4)

# print(len(localityToLGA))

In [20]:
def clean_lga_housing(housingPrice):
    with open("../localityToLGA.json", "r") as file:
        localityToLGAMap = json.load(file)


    def clean_communities(community: str):
        community = community.strip().lower()
        # Strip off any brackets at the end
        community = community.split("(")[0]
        # Rip off directions
        words = list(community.split())
        for i in range(len(words)):
            if words[i] in ("north", "south", "east", "west", "central"):
                words[i] = ""
        return ''.join(words)

    def localityToLGA(locality: str, localityToLGAMap: dict):
        return localityToLGAMap[clean_communities(locality)]


    housingPrice["LGA"] = housingPrice["Locality"].apply(lambda x: localityToLGA(x, localityToLGAMap))

# clean_lga_housing(housingPrice)
# housingPrice["LGA"]

In [21]:
# Preprocess
def clean_lga_all(communities, egm, housingPrice, offences):
    clean_lga_tables(communities=communities, egm=egm, offences=offences)
    clean_lga_housing(housingPrice)

clean_lga_all(communities, egm, housingPrice, offences)

In [27]:
allLGAs = {
    "communities": communities["LGA"].unique(),
    "egm": egm["LGA Name"].unique(),
    "housingPrice": housingPrice["LGA"].unique(),
    "offences": offences["Local Government Area"].unique()
}


commonLGAs = set(allLGAs["communities"]) \
    .intersection(set(allLGAs["egm"]))\
    .intersection(set(allLGAs["housingPrice"]))\
    .intersection(set(allLGAs["offences"]))

print("Check union of LGA: ", len(commonLGAs))
commonLGAs

Check union of LGA:  56


{'alpine',
 'ballarat',
 'banyule',
 'basscoast',
 'bawbaw',
 'bayside',
 'benalla',
 'boroondara',
 'brimbank',
 'campaspe',
 'cardinia',
 'casey',
 'centralgoldfields',
 'colacotway',
 'darebin',
 'eastgippsland',
 'frankston',
 'gleneira',
 'glenelg',
 'greaterbendigo',
 'greaterdandenong',
 'greatergeelong',
 'greatershepparton',
 'hobsonsbay',
 'horsham',
 'hume',
 'kingston',
 'knox',
 'latrobe',
 'macedonranges',
 'manningham',
 'maribyrnong',
 'maroondah',
 'melbourne',
 'melton',
 'merribek',
 'mildura',
 'mitchell',
 'monash',
 'mooneevalley',
 'moorabool',
 'morningtonpeninsula',
 'northerngrampians',
 'portphillip',
 'southgippsland',
 'stonnington',
 'surfcoast',
 'swanhill',
 'wangaratta',
 'wellington',
 'whitehorse',
 'whittlesea',
 'wodonga',
 'wyndham',
 'yarra',
 'yarraranges'}

In [23]:
# Preprocess for Housing Price: Grouping of LGA and imputation of data
# Replace '-' with NaN and unneeded/derived attributes
housingPrice = housingPrice.replace('-', 'NaN')
housingPrice = housingPrice.drop(columns=["Locality", "Change Percentage  13-23"])
for column in housingPrice.columns:
    if column == "LGA":
        continue
    housingPrice[column] = housingPrice[column].astype("float")

In [24]:
# Clean egm (mainly check for type)
for column in egm.columns:
    print(column, ":", egm[column].dtype)

LGA Name : object
Region : object
2011 : float64
2012 : float64
2013 : float64
2014 : float64
2015 : float64
2016 : float64
2017 : float64
2018 : float64
2019 : float64
2020 : float64


In [25]:
# Clean offenses
# Type check
for column in offences.columns:
    print(column, ":", offences[column].dtype)

# Drop NA for now, or ignore it cause won't use rate
# offences = offences.dropna()

# offences.isna().sum()

# Remove comma to convert it to float
offences["Offence Count"] = offences["Offence Count"].astype("string")
# offences["Rate per 100,000 population"] = offences["Rate per 100,000 population"].astype("string")

offences["Offence Count"] = offences["Offence Count"].apply(lambda x: x.replace(",", ""))
# offences["Rate per 100,000 population"] = offences["Rate per 100,000 population"].apply(lambda x: x.replace(",", ""))

offences["Offence Count"] = offences["Offence Count"].astype("float")
# offences["Rate per 100,000 population"] = offences["Rate per 100,000 population"].astype("float")

for column in offences.columns:
    print(column, ":", offences[column].dtype)

Year : int64
Year ending : object
Police Region : object
Local Government Area : object
Offence Count : object
Rate per 100,000 population : object
Year : int64
Year ending : object
Police Region : object
Local Government Area : object
Offence Count : float64
Rate per 100,000 population : object


In [26]:
# Clean communities
# Pick the features to process
# LGA (alp): Id, grouped by LGA
# Location (num): Derive x and y distance from Melbourne, and use average
# Area (km^2) (num) 

# Remove all % columns, or convert to raw if not available
# Remove 2007 columns (if don't want to extrapolate)
# Clean the <5 in all columns
# min, max, avg: Use all for now until correlation analysis

# Columns to drop - Only drop those that can't be grouped into LGA
# Compute population from population density
drop_cols_noGroup = ["Map reference", "Grid reference", "Travel time to GPO (minutes)", "Distance to GPO (km)", 
             "Primary Care Partnership", "Medicare Local", "DHS Area"]

# Columns to drop - Derived % or other derived attributes
drop_cols_derived = ["Commercial (%)", "Industrial (%)", "Residential (%)", 
                     "Rural (%)", "Other (%)", "Average persons per household", 
                     "Population Density", "Occupied private dwellings, %", 
                     "% dwellings which are public housing", 
                     ""]
# All the cols with the word age and %
# Don't use 2007 age groups, but derived the change in population


# Columns to transform - %

#Alpha columns to be transformed
#ABS remoteness category: Ordinal encoder